This notebook file is a RAG application that uses Ollama model to ask questions based on a pdf file. To execute this file, you need to download Ollama models from https://ollama.com/download. 

In [ ]:
MODEL = "llama2"

In [ ]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

#model.invoke("tell me a joke")

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("mlschool.pdf")
pages = loader.load_and_split()
pages

In [ ]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


In [ ]:


chain = prompt | model 

In [ ]:
# Use chain to invoke responses from model based on the format

chain.invoke(
    {
        "context": "I was called Mike",
        "question":"What is my name?"
    }
)

In [ ]:
# Use a vector store to store information from the pages as embeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [ ]:
retriever = vectorstore.as_retriever()

retriever.invoke("Machine learning")

In [ ]:
# Ask questions based on the documents provided 
from operator import itemgetter

chain = (

{
    "context": itemgetter("question") | retriever, 
    "question": itemgetter("question")

}
| prompt
| model
)

chain.invoke({"question": "What is machine learning?"})

In [ ]:
questions = [
    "What is the purpose of the course?",
    "How many hours of live sessions?",
    "How many coding assignments are there in the program?",
    "Is there a program certificate upon completion?",
    "What programming language will be used in the program?",
    "How much does the program cost?",
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({"question": question})}")
    print()



In [ ]:
# Alternative way of asking questions about the pdf
for s in chain.stream({"question": "How much does the program cost?"}):
    print(s, end="", flush=True)

In [ ]:
# Alternative way of asking questions about the pdf 
chain.batch([{"question": q} for q in questions])